In [77]:
import pandas as pd
from string import ascii_uppercase as alphabet
import pickle
from bs4 import BeautifulSoup
import requests

In [36]:
all_tables = pd.read_html('https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

In [57]:
#collecting all the tables with the information on all the Group Stages
all_tables[9]
all_tables[16]
all_tables[23]
all_tables[30]
all_tables[37]
all_tables[44]
all_tables[51]
all_tables[58]

,Pos,Teamvte,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Portugal,3,2,0,1,6,4,+2,6,Advanced to knockout stage
1,2,South Korea,3,1,1,1,4,4,0,4,Advanced to knockout stage
2,3,Uruguay,3,1,1,1,2,2,0,4,NaN
3,4,Ghana,3,1,0,2,5,7,−2,3,NaN


In [68]:
dict_table = {}

for letter, i in zip(alphabet, range(9,65,7)):
    #grabbing each tables and creating a dataframe for each group
    df= all_tables[i]
    df.rename(columns={df.columns[1]: 'Team'}, inplace=True)
    #Removing Qualification column
    df.pop('Qualification')
    dict_table[f'Group {letter}'] = df

In [70]:
dict_table.keys()

dict_keys(['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F', 'Group G', 'Group H'])

In [71]:
#View the table for Group A just to verify the right data was grabbed
dict_table['Group A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Netherlands,3,2,1,0,5,1,+4,7
1,2,Senegal,3,2,0,1,5,4,+1,6
2,3,Ecuador,3,1,1,1,4,3,+1,4
3,4,Qatar (H),3,0,0,3,1,7,−6,0


In [74]:
#All data in the dict_table that was created, put it in a file named output
with open('dict_table', 'wb') as output:
    pickle.dump(dict_table, output)

In [80]:
#Scrapping the Data from the previous world cups 

years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966,
        1970, 1974, 1978, 1982, 1986, 1990, 1994, 1998,
        2002, 2006, 2010, 2014, 2018]

#Steps needed to scrape data for 2014 WC
web = 'https://en.wikipedia.org/wiki/2014_FIFA_World_Cup'
response = requests.get(web)
content = response.text
soup = BeautifulSoup(content, 'lxml')
#grab all the matches into a list with lots of elements
matches = soup.find_all('div', class_='footballbox')


home = []
score = []
away = []


#look through the lists to grab the home team, score, and away team
for match in matches:
    home.append(match.find('th', class_='fhome').get_text())
    score.append(match.find('th', class_='fscore').get_text())
    away.append(match.find('th', class_='faway').get_text())
    
#dictionary
dict_football = {'home': home, 'score': score, 'away': away}

#creating the football dataframe
df_football = pd.DataFrame(dict_football)
#creating a column that will indicate the year of the given worldcup
df_football['year'] = '2014'

In [81]:
#preview of all the matches in the 2014 World Cup in Brazil
print(df_football)

            home         score          away  year
0        Brazil            3–1       Croatia  2014
1        Mexico            1–0      Cameroon  2014
2        Brazil            0–0        Mexico  2014
3      Cameroon            0–4       Croatia  2014
4      Cameroon            1–4        Brazil  2014
..           ...           ...           ...   ...
59  Netherlands   0–0 (a.e.t.)    Costa Rica  2014
60       Brazil            1–7       Germany  2014
61  Netherlands   0–0 (a.e.t.)     Argentina  2014
62       Brazil            0–3   Netherlands  2014
63      Germany   1–0 (a.e.t.)     Argentina  2014

[64 rows x 4 columns]


In [82]:
#Scrapping the data for all world cup up until 2018

years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966,
        1970, 1974, 1978, 1982, 1986, 1990, 1994, 1998,
        2002, 2006, 2010, 2014, 2018]

def get_matches(year):
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, 'lxml')
    #grab all the matches into a list with lots of elements
    matches = soup.find_all('div', class_='footballbox')


    home = []
    score = []
    away = []


    #look through the lists to grab the home team, score, and away team
    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())
    
    #dictionary
    dict_football = {'home': home, 'score': score, 'away': away}

    #creating the football dataframe
    df_football = pd.DataFrame(dict_football)
    #creating a column that will indicate the year of the given worldcup
    df_football['year'] = year
    return df_football

In [83]:
#Testing Function to see the results of the 2018 world cup
print(get_matches('2018'))

        home         score           away  year
0    Russia            5–0   Saudi Arabia  2018
1     Egypt            0–1        Uruguay  2018
2    Russia            3–1          Egypt  2018
3   Uruguay            1–0   Saudi Arabia  2018
4   Uruguay            3–0         Russia  2018
..       ...           ...            ...   ...
59   Russia   2–2 (a.e.t.)        Croatia  2018
60   France            1–0        Belgium  2018
61  Croatia   2–1 (a.e.t.)        England  2018
62  Belgium            2–0        England  2018
63   France            4–2        Croatia  2018

[64 rows x 4 columns]


In [84]:
fifa = [get_matches(year) for year in years]
#join all the tables from the fifa list that has all the tables
df_fifa = pd.concat(fifa, ignore_index =True)
df_fifa.to_csv('fifa_worldcup_historical_data.csv', index=False)


In [85]:
#Fixtures for 2022 Worldcup 
df_fixture22 = get_matches(2022)

In [87]:
#Save as a csv
df_fixture22.to_csv('fifa_worldcup2022_fixture.csv', index=False)